In [ ]:
import os
import cwt
from cwt import auth

url = os.environ.get('WPS_URL', 'https://aims2.llnl.gov/wps')
token = os.environ.get('WPS_TOKEN', None)
verify = bool(os.environ.get('WPS_VERIFY', True))
timeout = float(os.environ.get('WPS_TIMEOUT', 120))

In [ ]:
token = auth.TokenAuthenticator(token, key='COMPUTE_TOKEN', value='{}')

client = cwt.WPSClient(url, auth=token, verify=verify)

In [ ]:
d0 = cwt.Domain(time=('1979-06-01', '1980-06-01'), lat=(-45, 45))

v0 = cwt.Variable('http://aims3.llnl.gov/thredds/dodsC/cmip5_css02_data/cmip5/output1/CMCC/CMCC-CM/amip/3hr/atmos/3hr/r1i1p1/pr/1/pr_3hr_CMCC-CM_amip_r1i1p1_197901010130-197912312230.nc', 'pr')
v1 = cwt.Variable('http://aims3.llnl.gov/thredds/dodsC/cmip5_css02_data/cmip5/output1/CMCC/CMCC-CM/amip/3hr/atmos/3hr/r1i1p1/pr/1/pr_3hr_CMCC-CM_amip_r1i1p1_198001010130-198008291630.nc', 'pr')

In [ ]:
aggregate = client.CDAT.aggregate(v0, v1)

subset = client.CDAT.subset(aggregate, domain=d0)

In [ ]:
client.execute(subset)

subset.wait(timeout=timeout)

In [ ]:
import xarray as xr
import hashlib

ds = xr.open_mfdataset([x.uri for x in subset.output], chunks={'time': 100}, combine='by_coords')

digest = hashlib.sha256(ds.pr.values.tobytes()).hexdigest()

assert digest == 'c525ec07a6f535d416fdefc3def2b3508faa661f4d7e2bafb0b2ac2f80d8156f', f'missmatch digest {digest}'